In [18]:
# ── cell: 1 ──
# 📁 Prepare Workspace


MODEL_NAME= "facebook_mbart-large-50-many-to-many-mmt" # "fewshot_gemini" #"agemagician_mlong-t5-tglobal-large"  #"UBC-NLP_AraT5-base"
MODEL_TYPE= "fine_tuned_model"

import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) <= 5)

# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    !mkdir -p /kaggle/working/evaluation/ar_style_classifier/
    !cp -r /kaggle/input/binary-style-classifier-models-21-authors/working/* /kaggle/working/evaluation/ar_style_classifier/
    !cp -r /kaggle/input/ast-fine-tuned-models-generations/results/results/* /kaggle/working/models/fine_tuned_model/results/

    !mkdir -p /kaggle/working/models/baseline_model/results/fewshot_gemini
    !cp -r /kaggle/input/ast-fine-tuned-models-generations/baseline_model/baseline_model/results/fewshot_gemini/generation_log.json /kaggle/working/models/baseline_model/results/fewshot_gemini
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")


# !cp -r /kaggle/input/binary-style-classifier-models-21-authors/working/* /kaggle/working/evaluation/ar_style_classifier/
!cp -r /kaggle/input/ast-fine-tuned-models-generations/results/results/* /kaggle/working/models/fine_tuned_model/results/



# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

❌ '/kaggle/working' is not empty — contains 10 items.
/kaggle/working
config.py  evaluation	models	 __pycache__	  utils
data	   __init__.py	prompts  run_pipeline.py


In [19]:
!cp -r /kaggle/input/arageneval2025-task1-finaldataset/PublicDataFinalPhaseTask1.xlsx /kaggle/working/data

# data/PublicDataFinalPhaseTask1.xlsx

In [20]:
!ls -R /kaggle/working/models/{MODEL_TYPE}/results/
!ls -R /kaggle/working/models/{MODEL_TYPE}/results/{MODEL_NAME}

/kaggle/working/models/fine_tuned_model/results/:
agemagician_mlong-t5-tglobal-large	  sultan_ArabicT5-49GB-base
facebook_mbart-large-50-many-to-many-mmt  UBC-NLP_AraT5-base
google_mt5-small			  UBC-NLP_AraT5v2-base-1024
moussaKam_AraBART			  UBC-NLP_AraT5v2-base-1024_with_lora
moussaKam_AraBART_with_lora

/kaggle/working/models/fine_tuned_model/results/agemagician_mlong-t5-tglobal-large:
generation_log.json  generation_log_v1.json  generation_log_v2.json

/kaggle/working/models/fine_tuned_model/results/facebook_mbart-large-50-many-to-many-mmt:
generation_log.json	  generation_log_v2.json
generation_log_test.json  generation_log_v3.json

/kaggle/working/models/fine_tuned_model/results/google_mt5-small:
generation_log_ERROR.json  generation_log.json

/kaggle/working/models/fine_tuned_model/results/moussaKam_AraBART:
generation_log.json

/kaggle/working/models/fine_tuned_model/results/moussaKam_AraBART_with_lora:
generation_log.json	  generation_log_v1.json
generation_log_test.json  gene

# Dependencies

In [21]:
!pip install --upgrade transformers==4.41.2 datasets==2.19.1 scikit-learn wandb python-dotenv evaluate --quiet
!pip install -U peft==0.11.1 --quiet

In [22]:
import sys
import os
import torch

project_root = os.getcwd()  # Should be /content/drive/MyDrive/AraGenEval2025
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from config import Config

print(Config.VAL_FILE)

Project root added to sys.path: /kaggle/working
Using device: cuda
data/AuthorshipStyleTransferVal.xlsx


# Mapping

In [23]:
# ── cell: 3 ──
# 🗺 Author Mappings
author2id = {
    "أحمد أمين": 0,  "أحمد تيمور باشا": 1,  "أحمد شوقي": 2,
    "أمين الريحاني": 3,  "ثروت أباظة": 4,  "جبران خليل جبران": 5,
    "جُرجي زيدان": 6,  "حسن حنفي": 7,  "روبرت بار": 8,
    "سلامة موسى": 9,  "طه حسين": 10, "عباس محمود العقاد": 11,
    "عبد الغفار مكاوي": 12, "غوستاف لوبون": 13,  "فؤاد زكريا": 14,
    "كامل كيلاني": 15,  "محمد حسين هيكل": 16,  "نجيب محفوظ": 17,
    "نوال السعداوي": 18, "ويليام شيكسبير": 19, "يوسف إدريس": 20
}
id2author   = {v: k for k, v in author2id.items()}
id2english  = {
    0:"Ahmed_Amin",    1:"Ahmad_Taymour_Basha",   2:"Ahmed_Shawqi",
    3:"Ameen_Rihani",  4:"Tharwat_Abaza",         5:"Gibran_Khalil_Gibran",
    6:"Jurji_Zaydan",  7:"Hassan_Hanifi",         8:"Robert_Barr",
    9:"Salama_Moussa",10:"Taha_Hussein",         11:"Abbas_Al-Aqqad",
   12:"AbdelGhaffar_Makawi",13:"Gustave_Lebon",    14:"Fouad_Zakaria",
   15:"Kamel_Kilani",16:"Mohamed_Hosseini_Hekal",17:"Naguib_Mahfouz",
   18:"Nawal_El_Saadawi",19:"William_Shakespeare",20:"Youssef_Edrees"
}


# Load Dataset

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

def load_and_sample_dataset(
    file_path,
    sample_mode="all",    # Options: "all", "random", "stratified"
    sample_size=None,     # For "random"/"stratified": float (proportion) or int (absolute count)
    random_state=42
):
    """
    Load dataset from Excel and apply sampling.

    Args:
        file_path (str): Path to dataset file.
        sample_mode (str): "all", "random", or "stratified".
        sample_size (float|int): Proportion (0-1) or count for sampling.
        random_state (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Sampled dataset.
    """
    print(f"📂 Loading dataset from: {file_path}")
    df = pd.read_excel(file_path, engine='openpyxl')

    if sample_mode == "all":
        sampled_df = df.copy()
        print(f"✅ Loaded full dataset with {len(sampled_df)} samples.")

    elif sample_mode == "random":
        if sample_size is None:
            raise ValueError("For 'random' mode, SAMPLE_SIZE must be set.")
        sampled_df = df.sample(
            n=sample_size if isinstance(sample_size, int) else int(len(df) * sample_size),
            random_state=random_state
        )
        print(f"✅ Randomly sampled {len(sampled_df)} samples.")

    elif sample_mode == "stratified":
        if sample_size is None or not (0 < sample_size < 1):
            raise ValueError("For 'stratified' mode, SAMPLE_SIZE must be a proportion between 0 and 1.")
        sampled_df, _ = train_test_split(
            df,
            train_size=sample_size,
            stratify=df['author'],
            random_state=random_state
        )
        print(f"✅ Stratified sampled {len(sampled_df)} samples (author distribution preserved).")

    else:
        raise ValueError(f"Invalid sample_mode: {sample_mode}")

    del df
    gc.collect()

    return sampled_df

In [25]:
# === CONFIG ===
SAMPLE_MODE = "stratified"  # "all", "random", "stratified"
SAMPLE_SIZE = 0.1           # 10% for stratified or random. Set to None for full dataset.

# === LOAD AND SAMPLE ===
val_df = load_and_sample_dataset(
    "data/PublicDataFinalPhaseTask1.xlsx",
    # Config.VAL_FILE,
    # sample_mode=SAMPLE_MODE,
    # sample_size=SAMPLE_SIZE
)

# Expect columns: id, text_in_msa
val_df = val_df.rename(columns={"id":"sample_id","text_in_msa":"neutral_text"})
print(f"✅ Loaded {len(val_df)} neutral texts.")

print("Validation shape:", val_df.shape)
print("Authors in Val:", val_df['author'].nunique())

# train_df.head()

📂 Loading dataset from: data/PublicDataFinalPhaseTask1.xlsx
✅ Loaded full dataset with 8413 samples.
✅ Loaded 8413 neutral texts.
Validation shape: (8413, 3)
Authors in Val: 21


## Sliding Window

In [26]:
# ── cell: 6 ──
# 🪓 Sliding Window Chunking Utility
from transformers import AutoTokenizer
import torch

def get_chunks(text, tokenizer, max_len=512, stride=256):
    return tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_tensors="pt"
    )

def device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Model 

In [27]:
# ── cell: 7 ──
# 🔧 Load Classifier Function
from transformers import AutoModelForSequenceClassification

def load_binary_classifier(author_id):
    path = f"evaluation/ar_style_classifier/author_SC_{author_id}"
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForSequenceClassification.from_pretrained(path)
    model.to(device())
    model.eval()
    return tokenizer, model

## Preparations

In [28]:
# ── cell: 5 ──
# 📝 Load Generation Log
import json
with open(f"models/{MODEL_TYPE}/results/{MODEL_NAME}/generation_log_test.json","r", encoding="utf-8") as f:
    gen = json.load(f)
# gen: list of {id, author, output, ground_truth}
log_df = pd.DataFrame(gen)
log_df = log_df.rename(columns={"id":"sample_id","output":"generated_text","author":"true_author"})
print(f"✅ Loaded {len(log_df)} generated entries.")

# Merge neutral and generated
df = pd.merge(log_df, val_df, on="sample_id", how="left")
df["true_id"] = df["true_author"].map(author2id)
print(df.head())

✅ Loaded 8413 generated entries.
   sample_id       true_author  \
0          1         روبرت بار   
1          2       جُرجي زيدان   
2          3        نجيب محفوظ   
3          4  جبران خليل جبران   
4          5        يوسف إدريس   

                                         text_in_msa  \
0  إن كان عددها هائلًا إلى حدٍّ يتطلب جهدًا من ال...   
1  قال: "هذا هو سلمان الذي جاءنا بالأخبار اليقيني...   
2  حدَّق إليها بنظرة فاحصة وقاسية، فرآها تتسم بال...   
3  انصرف لويص من أمام زعيم القبيلة، متوجهًا إلى م...   
4  إنَّه لمن العبث الاستطراد في توضيح أو تصوير خط...   

                                      generated_text  \
0  إن كان عددها هائلًا إلى حدٍّ ما، فلن نحصِّله، ...   
1  قال: هذا هو سلمان الذي جاءنا بالأخبار اليقينية...   
2  وحدَّق إليها بنظرةٍ فاحصةٍ قاسية، فرآها مُكبرة...   
3  انصرف لويص من أمام زعيم القبيلة، وذهب إلى مرقد...   
4  ومن العبث أن أستغرق في بيان أو تصوير خطورة ما ...   

                                        neutral_text            author  \
0  إن كان عدده

# Prediction

In [29]:
# ── cell: 8 ──
# 🔍 Probability Prediction (in-style)
import numpy as np

# def predict_instyle_prob(text, tokenizer, model):
#     enc = get_chunks(text, tokenizer)
#     enc = {k: v.to(device()) for k,v in enc.items()}
#     with torch.no_grad():
#         logits = model(**enc).logits  # [chunks, 2]
#         probs = torch.softmax(logits, dim=-1)  # same shape
#     instyle_probs = probs[:,1]  # author style class
#     return instyle_probs.mean().item()


VALID_KEYS = {"input_ids", "attention_mask", "token_type_ids"}

def predict_instyle_prob(text, tokenizer, model):
    enc = get_chunks(text, tokenizer)
    enc = {k: v.to(device()) for k,v in enc.items() if k in VALID_KEYS}
    with torch.no_grad():
        logits = model(**enc).logits  # Now works
        probs = torch.softmax(logits, dim=-1)
    return probs[:,1].mean().item() # author style class


In [30]:
# # ── cell: 9 ──
# # 📊 Style Evaluation Loop
# from tqdm import tqdm
# results = []
# for author_id in range(21):
#     tokenizer, model = load_binary_classifier(author_id)
#     for row in tqdm(df.itertuples(), desc=f"Author {author_id}"):
#         px = predict_instyle_prob(row.neutral_text, tokenizer, model)
#         py = predict_instyle_prob(row.generated_text, tokenizer, model)
#         diff = py - px
#         success = diff > 0
#         results.append({
#             "sample_id": row.sample_id,
#             "true_id": row.true_id,
#             "classifier_id": author_id,
#             "neutral_prob": px,
#             "generated_prob": py,
#             "diff": diff,
#             "success": int(success)
#         })
# results_df = pd.DataFrame(results)



# ── cell: 9 ──
# 📊 Style Evaluation Loop (per‑author filtering)
from tqdm import tqdm

results = []

for author_id in range(21):
    # 1) select only the samples meant for this author
    author_df = df[df["true_id"] == author_id]

    # 2) load that author’s classifier
    tokenizer, model = load_binary_classifier(author_id)

    # 3) run only their own samples
    for row in tqdm(author_df.itertuples(), desc=f"Author {author_id}"):
        px   = predict_instyle_prob(row.neutral_text,   tokenizer, model)
        py   = predict_instyle_prob(row.generated_text, tokenizer, model)
        diff = py - px
        success = int(diff > 0)

        results.append({
            "sample_id":      row.sample_id,
            "true_id":        row.true_id,
            "classifier_id":  author_id,
            "neutral_prob":   px,
            "generated_prob": py,
            "diff":           diff,
            "success":        success
        })

# build DataFrame after the loop
results_df = pd.DataFrame(results)

Author 0: 597it [00:31, 18.87it/s]
Author 1: 147it [00:08, 18.01it/s]
Author 2: 55it [00:03, 16.61it/s]
Author 3: 624it [00:34, 17.97it/s]
Author 4: 191it [00:10, 17.94it/s]
Author 5: 240it [00:11, 20.13it/s]
Author 6: 570it [00:32, 17.71it/s]
Author 7: 1004it [00:53, 18.66it/s]
Author 8: 512it [00:29, 17.34it/s]
Author 9: 282it [00:13, 21.32it/s]
Author 10: 538it [00:23, 23.30it/s]
Author 11: 500it [00:22, 22.15it/s]
Author 12: 464it [00:22, 20.79it/s]
Author 13: 358it [00:15, 23.79it/s]
Author 14: 294it [00:12, 24.34it/s]
Author 15: 110it [00:07, 15.43it/s]
Author 16: 492it [00:23, 21.14it/s]
Author 17: 343it [00:18, 19.03it/s]
Author 18: 383it [00:20, 19.11it/s]
Author 19: 360it [00:20, 17.26it/s]
Author 20: 349it [00:14, 23.73it/s]


# Save results

In [31]:
save_dir = f"evaluation/ar_style_classifier/results/{MODEL_NAME}"
!mkdir -p {save_dir}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
# ── cell: 10 ──
# 💾 Save Results and Metrics
results_df.to_csv(
    f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_evaluation_results_.csv",
    index=False
)

# 🧾 Aggregate metrics per classifier
metrics = results_df.groupby("classifier_id").agg(
    accuracy=("success", "mean"),
    avg_diff=("diff", "mean"),
    count=("success", "size")
).reset_index()

# 🌐 Compute overall metrics across all classifiers and samples
overall_accuracy = results_df["success"].mean()
overall_avg_diff = results_df["diff"].mean()

# Create an overall-metrics DataFrame row
overall_row = pd.DataFrame([{
    "classifier_id": -1,
    "accuracy": overall_accuracy,
    "avg_diff": overall_avg_diff,
    "count": len(results_df)
}])

# Concatenate to include overall stats
metrics = pd.concat([metrics, overall_row], ignore_index=True)

# 🖨 Print overall metrics
print(f"Overall Style-Transfer Accuracy: {overall_accuracy:.4f}")
print(f"Overall Average Diff (strength): {overall_avg_diff:.4f}")

# 💾 Save both per-author and overall metrics
metrics.to_csv(
    f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_metrics_per_author_and_overall.csv",
    index=False
)

print("✅ Saved evaluation and per-author metrics CSVs (including overall metric).")


Overall Style-Transfer Accuracy: 0.8610
Overall Average Diff (strength): 0.2932
✅ Saved evaluation and per-author metrics CSVs (including overall metric).


In [33]:
# ── cell: 11 ──
# 📈 Per-Sample Summary: Identify strongest style match and overall success rate
# For each sample, find the classifier with highest generated_prob
top_matches = results_df.loc[results_df.groupby('sample_id')['generated_prob'].idxmax()]
# top_matches = top_matches.merge(id2author.to_frame('classifier_id').reset_index(), on='classifier_id')
id2author_df = pd.DataFrame(list(id2author.items()), columns=['classifier_id', 'predicted_author'])
top_matches = top_matches.merge(id2author_df, on='classifier_id')
top_matches = top_matches.rename(columns={
    'classifier_id': 'predicted_id',
    'index': 'predicted_author'
})
# Calculate overall success: did the true author classifier succeed?
true_success = results_df[
    (results_df['classifier_id'] == results_df['true_id'])
].groupby('sample_id')['success'].first().reset_index()

# Merge for overview
overview = top_matches.merge(true_success, on='sample_id')

overview.to_csv(f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_evaluation_per_sample_overview.csv", index=False)
print("✅ Saved per-sample overview CSV: strongest match and true-author success.")

# 💡 Note: Probabilities are averaged across all chunks of a sample, so each sample has a single neutral_prob and generated_prob per classifier.

✅ Saved per-sample overview CSV: strongest match and true-author success.


In [34]:
# ── cell: 12 ──
# 📦 Package Results Archive
import os, subprocess

zip_name = f"ar_style_classifier_results_{MODEL_NAME}.zip"
results_dir = os.path.join("evaluation", "ar_style_classifier", "results", MODEL_NAME)

# Run compression
!cd evaluation/ar_style_classifier && zip -r {zip_name} results/{MODEL_NAME}

print(f"✅ Created archive: evaluation/ar_style_classifier/{zip_name}")

# !cd evaluation/ar_style_classifier && zip -r ar_style_classifier_results_{MODEL_NAME}.zip results/{MODEL_NAME}

  adding: results/facebook_mbart-large-50-many-to-many-mmt/ (stored 0%)
  adding: results/facebook_mbart-large-50-many-to-many-mmt/ar_bsc_metrics_per_author_and_overall.csv (deflated 47%)
  adding: results/facebook_mbart-large-50-many-to-many-mmt/ar_bsc_evaluation_results_.csv (deflated 60%)
  adding: results/facebook_mbart-large-50-many-to-many-mmt/ar_bsc_evaluation_per_sample_overview.csv (deflated 65%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Created archive: evaluation/ar_style_classifier/ar_style_classifier_results_facebook_mbart-large-50-many-to-many-mmt.zip


In [36]:
# # --- Inference on full validation texts ---
# def predict_full_text(text):
#     enc = tokenizer(
#         text,
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#         stride=256,
#         return_overflowing_tokens=True,
#         return_tensors="pt"
#     ).to(device)
#     logits = model(**enc).logits  # [num_chunks, num_labels]
#     avg_logits = logits.mean(dim=0)
#     probs = torch.softmax(avg_logits, dim=0)
#     return torch.argmax(probs).item(), probs.cpu().tolist()

In [37]:
# # --- Evaluate across validation set ---
# preds, refs = [], []
# for example in val_df.itertuples():
#     pred, _ = predict_full_text(example.text_in_author_style)
#     preds.append(pred)
#     refs.append(author2id[example.author])

In [38]:
# from sklearn.metrics import accuracy_score, f1_score
# print("Validation Accuracy:", accuracy_score(refs, preds))
# print("Validation F1‑macro:", f1_score(refs, preds, average="macro"))